# Lotka–Volterra predator-prey model with Gillespie simulation
The modeling done here demonstrates Particle's ability to model a dynamic system.  The example is from ecology, but applies readily to problems in econometrics, finance, chemistry, and systems biology.

In [ ]:
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

from particle import Exponential
from particle import Dist
from particle import Discrete
from particle import Condition
from particle import MH
from particle import SingleSite

## Background

### Lotka-Volterra model
This is a simple dynamic model of the populations of a single predator species and a single prey species in an ecology that does contain any other species.  Three basic events occur in this system:

1. A prey spawns a new prey
2. A predator eats a prey and immediately spawns a new predator
3. A predator dies

The population of these species in time depends on the rates at which these events occur.  If predators eat the prey too often, the prey die out then the predators die out. If the predation occurs to rarely of course the prey population explodes.  Of course the rates that these events occurs at a particular point in time depends on the populations at a specific point in time.

### Gillespie algorithm

The Gillespie algorithm is an algorithm for modeling this type of systems as a discrete state continuous time Markov process.  This means the events' occurance rates are used to calculate the probability an event will occur at a specific instant in time (hazard rate), then forward simulating using the hazard rates.

## Build the program to select from three hypotheses
There are three rate parameters that determine the rates of the three events.  Typically one puts independent Gamma priors on these.  But to simplicify the inference task, we are just going to specify three discrete parameter configurations and assign equal prior probability to each configuration.

In [ ]:
class LV(Dist):
    """
    Gillespie simulation of the Lotka–Volterra predator-prey model.
    """
    def __init__(self, transitions, initial, N, select_hyp="none", t0=0.0):
        """
        Create the Gillespie object.

        Args:
            transitions(dict): A dictionary with keys "prey" and "pred", and each element
                is a list containing the row of the transition matrix.
            initial(dict): A dictionary with keys 'prey' and 'predator' giving the initial
                counts for each species at the start of the sim.
            N(int): Number of desired simulation iterations.
            select_hyp(str): Default is "none", wherein a uniform prior is assigned to the
                three hypotheses.  Otherwise, "H1", "H2", or "H3" concentrates all prior
                probability on one hypothesis (useful in forward simulation).
            t0(float): initial time point.
        """
        self.transitions = transitions
        self.select_hyp = select_hyp
        self.initial = initial
        self.times = [t0]
        self.trajectory = {'prey': [initial['prey']], 'pred': [initial['pred']]}
        self.N = N

    def get_hazards(self, ecology, theta):  # hazard function, note in this case it is constant in time
        """
        Compute the hazard function given the current states.  Note that in this
            model the function depends only on state, not time, though this is not
            the case in general. "spawn_prey" represents the even of a prey being born,
            "prey2pred" represents a predator consuming a new prey and consequently spawning
            a new predator, "pred_dies" represents the death of a predator.

        args:
            ecology(dict): a dictionary containing species counts.
            theta(dict): A dictionary where keys represent events (reactions) and
                values are the location hyperparameters for rate parameter constants
                corresponding to those events.
        """

        return {
            'spawn_prey': theta['spawn_prey'] * ecology['prey'],
            'prey2pred': theta['prey2pred'] * ecology['prey'] * ecology['pred'],
            'pred_dies': theta['pred_dies'] * ecology['pred']
        }

    def run(self, env):
        ecology = deepcopy(self.initial)
        theta_hypotheses = {
            'H1': {'spawn_prey': 1, 'prey2pred': 0.005, 'pred_dies': 0.6},
            'H2': {'spawn_prey': .9, 'prey2pred': 0.004, 'pred_dies': 0.4},
            'H3': {'spawn_prey': 1.5, 'prey2pred': 0.01, 'pred_dies': 0.8}
        }
        if self.select_hyp is not "none":
            theta_dex = env.bind("theta", Discrete({'H1': 0.0, 'H2': 0.0, 'H3': 0.0, self.select_hyp: 1.0}))
        else:
            theta_dex = env.bind("theta", Discrete({'H1': 1 - 0.6666, 'H2': 0.3333, 'H3': 0.3333}))

        theta = theta_hypotheses[theta_dex]

        t = self.times[0]
        for i in range(1, self.N):
            hazards = self.get_hazards(ecology, theta)
            transition = self.transitions[env.bind("event_{}".format(i), Discrete(hazards))]
            t = t + env.bind("t_{}".format(i), Exponential(sum(hazards.values())))

            ecology['prey'] += transition[0]
            ecology['pred'] += transition[1]
            # Enforce only positive integers
            ecology['prey'] = max(0, ecology['prey'])
            ecology['pred'] = max(0, ecology['pred'])

            self.trajectory['prey'].append(ecology['prey'])
            self.trajectory['pred'].append(ecology['pred'])
            self.times.append(t)

## Simulate 100 events for hypothesis 1

### Prepare the transition matrix and initial state

In [1]:
import numpy as np

In [2]:
Pre = np.array([[1, 0], [1, 1], [0, 1]])
Post = np.array([[2, 0], [0, 2], [0, 0]])
transition_mat = np.transpose(Post - Pre)
events = ['spawn_prey', 'prey2pred', 'pred_dies']
transitions = {events[k]: transition_mat[:, k] for k in range(len(events))}

In [3]:
transitions

{'spawn_prey': array([1, 0]),
 'prey2pred': array([-1,  1]),
 'pred_dies': array([ 0, -1])}

### Run the simulation and examine the trace
Running for 10000 data points.

In [ ]:
sim = LV(transitions, {'prey': 50, 'pred': 100}, 10000, select_hyp="H1")
trace = sim.generate()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
ax.set_ylabel("count")
ax.set_xlabel("time")
for v in ['prey', 'pred']:
    ax.plot(sim.times, sim.trajectory[v], label=v)

ax.legend()
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
ax.set_ylabel("predators")
ax.set_xlabel("prey")
ax.plot(sim.trajectory['prey'], sim.trajectory['pred'])
plt.show()

## Attempt to infer the hypothesis on a small set of events

We would like to use this model for infering the rate parameters.  There are three sets of rate parameters, H1, H2, and H3.  Assuming we didn't know which was the true set, we would condition on observed events and the time they occurred, and infer the posterior probability a given hypothesis was the true hypothesis.

In this example I will forward simulate from H1.  If inference is working, H1 should be the MAP (have the highest posterior probability).  

I limit the simulated evidence to only 20 events, few enough that there is still some uncertainty as to which hypothesis is true.

In [ ]:
N = 20  # Number of events
num_samp = 1000
sim = LV(transitions, {'prey': 50, 'pred': 100}, N, select_hyp="H1")
trace = sim.generate()
evidence = {k: v for k, v in trace.value.items() if k not in 'theta'}

In [ ]:
conditioned_sim = Condition(LV,evidence)(transitions, {'prey': 50, 'pred': 100}, N)
trace = conditioned_sim.generate()
solver = MH(conditioned_sim, SingleSite(), num_samples=num_samp, start_trace=trace)
output = solver.run()

Get hypothesis marginals

In [ ]:
{'H{}'.format(i): [output.results[i].value['theta'] for i in range(len(output.results))].count('H{}'.format(i))/num_samp for i in [1, 2, 3]}

While there is still some entropy in the posterior, the correct hypothesis still emerges as the best estimate.